# RDF Conversions

This notebook demonstrates how to [convert](https://nexus-forge.readthedocs.io/en/latest/interaction.html#converting) data in any RDF format (n3, Turtle, ...) to Resources and vice-versa. 

In [1]:
from kgforge.core import KnowledgeGraphForge

A configuration file is needed in order to create a KnowledgeGraphForge session. A configuration can be generated using the notebook [00-Initialization.ipynb](00%20-%20Initialization.ipynb).

In [2]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

## Imports

In [3]:
from kgforge.core import Resource

## Resource to rdflib.Graph

In [17]:
jane = Resource(type="Person", name="Jane Doe")

In [18]:
forge.register(jane)

<action> _register_one
<succeeded> True


In [19]:
graph = forge.as_graph(jane)

In [20]:
len(graph) == 2

True

In [21]:
for triple in graph:
    print(triple)

(rdflib.term.URIRef('https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/e75061b0-8054-47e1-965b-260b3af3204f'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://schema.org/Person'))
(rdflib.term.URIRef('https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/e75061b0-8054-47e1-965b-260b3af3204f'), rdflib.term.URIRef('http://schema.org/name'), rdflib.term.Literal('Jane Doe'))


In [22]:
graph_store_metadata = forge.as_graph(jane, store_metadata=True)
len(graph_store_metadata)

14

In [23]:
for triple in graph_store_metadata:
    print(triple)

(rdflib.term.URIRef('https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/e75061b0-8054-47e1-965b-260b3af3204f'), rdflib.term.URIRef('https://bluebrain.github.io/nexus/vocabulary/outgoing'), rdflib.term.URIRef('https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/e75061b0-8054-47e1-965b-260b3af3204f/outgoing'))
(rdflib.term.URIRef('https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/e75061b0-8054-47e1-965b-260b3af3204f'), rdflib.term.URIRef('https://bluebrain.github.io/nexus/vocabulary/constrainedBy'), rdflib.term.URIRef('https://bluebrain.github.io/nexus/schemas/unconstrained.json'))
(rdflib.term.URIRef('https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/e75061b0-8054-47e1-965b-260b3af3204f'), rdflib.term.URIRef('https://bluebrain.github.io/nexus/vocabulary/schemaProject'), rdflib.term.URIRef('https://sandbox.bluebrainnexus.io/v1/projects/github-users/mfsy'))
(rdflib.term.URIRef('https://sandbox.bluebrainnexus.io/v1/resources/

## rdflib.Graph to Resource

In [24]:
#data in n3 format
data = '''
@prefix schema: <http://schema.org/> .
[] a schema:Person;
     schema:name "Jane Doe" ;
     schema:knows <http://example.org/JohnDoe>;
     schema:affiliation <https://www.grid.ac/institutes/grid.5333.6>.

<http://example.org/JohnDoe> a schema:Person;
     schema:name "John Doe" .
     
<https://www.grid.ac/institutes/grid.5333.6> a schema:Organization;
     schema:name "EPFL" .
'''

In [25]:
import rdflib
graph = rdflib.Graph()
graph.parse(data=data, format="n3")
len(graph)

8

In [26]:
resources = forge.from_graph(graph)

In [27]:
len(resources) == 3

True

### Apply a JSON-LD Frame

In [28]:
frame = {
        "@type": [ 'http://schema.org/Person'],
        "@embed": True
}  
resources = forge.from_graph(data=graph, frame= frame)

In [29]:
len(resources) == 2

True

In [30]:
forge.as_jsonld(resources)

[{'@context': {'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
   'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
   'schema': 'http://schema.org/',
   'xsd': 'http://www.w3.org/2001/XMLSchema#'},
  '@type': 'schema:Person',
  'schema:affiliation': {'@id': 'https://www.grid.ac/institutes/grid.5333.6',
   '@type': 'schema:Organization',
   'schema:name': 'EPFL'},
  'schema:knows': {'@id': 'http://example.org/JohnDoe',
   '@type': 'schema:Person',
   'schema:name': 'John Doe'},
  'schema:name': 'Jane Doe'},
 {'@context': {'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
   'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
   'schema': 'http://schema.org/',
   'xsd': 'http://www.w3.org/2001/XMLSchema#'},
  '@id': 'http://example.org/JohnDoe',
  '@type': 'schema:Person',
  'schema:name': 'John Doe'}]

### Select a Type

In [31]:
_type= ["http://schema.org/Organization"]
resources = forge.from_graph(data=graph, type= _type)

In [32]:
isinstance(resources, Resource)

True

In [33]:
print(resources)

{
    context:
    {
        rdf: http://www.w3.org/1999/02/22-rdf-syntax-ns#
        rdfs: http://www.w3.org/2000/01/rdf-schema#
        schema: http://schema.org/
        xsd: http://www.w3.org/2001/XMLSchema#
    }
    id: https://www.grid.ac/institutes/grid.5333.6
    type: schema:Organization
    "schema:name": EPFL
}


In [34]:
isinstance(resources, Resource)

True

In [35]:
epfl_json = {'id': 'https://www.grid.ac/institutes/grid.5333.6',
  'type': 'schema:Organization',
  'schema:name': 'EPFL'}

In [36]:
epfl_json == forge.as_json(resources)

True

### Use the forge Model context

In [37]:
_type= ["http://schema.org/Organization"]
resources = forge.from_graph(data=graph, type= _type, use_model_context=True)

In [38]:
isinstance(resources, Resource)

True